# ARC Tools
## TS Guesses
Enter smiles of reactants and products. Write ``None`` if needed.

In [3]:
from arc.imports import settings
from arc.job.adapters.common import all_families_ts_adapters, ts_adapters_by_rmg_family
from arc.plotter import draw_structure
from arc.reaction import ARCReaction
from arc.rmgdb import make_rmg_database_object, load_families_only
from arc.species import ARCSpecies
from arc.species.converter import xyz_to_str
from arc.scheduler import Scheduler

%matplotlib inline

In [1]:
r1_smiles = '[OH]'
r2_smiles = 'CCC'

p1_smiles = 'O'
p2_smiles = '[CH2]CC'

In [2]:
# influences the number of guesses from heuristics and GCN
dihedral_increment = 90

In [4]:
rmgdb = make_rmg_database_object()
load_families_only(rmgdb)

In [5]:
def generate_species_from_smiles(smiles):
    return ARCSpecies(label=smiles, smiles=smiles)

In [6]:
r_species, p_species = list(), list()
for r_smiles in [r1_smiles, r2_smiles]:
    if r_smiles is not None:
        r_species.append(generate_species_from_smiles(r_smiles))
for p_smiles in [p1_smiles, p2_smiles]:
    if p_smiles is not None:
        p_species.append(generate_species_from_smiles(p_smiles))

rxn = ARCReaction(r_species=r_species, p_species=p_species)
rxn.ts_species = ARCSpecies(label='TS', is_ts=True)
rxn.determine_family(rmg_database=rmgdb)

print(f'Considering the following reaction:\n\n{rxn.label}\n\nReaction family: {rxn.family.label}')

Considering the following reaction:

[OH] + CCC <=> O + [CH2]CC

Reaction family: H_Abstraction


In [7]:
sched = Scheduler(project='TS_search_1',
                  species_list=rxn.r_species + rxn.p_species,
                  rmg_database=rmgdb,
                  conformer_level='wb97xd/Def2TZVP',
                  opt_level='wb97xd/Def2TZVP',
                  sp_level='wb97xd/Def2TZVP',
                  freq_level='wb97xd/Def2TZVP',
                  ess_settings=None,
                  testing=True,
                  project_directory='/home/alon/Dropbox (MIT)/runs/TS_search_1',
                 )
sched.job_dict = {'TS': {'tsg': {}}}

In [8]:
tsg_index = 0
ts_adapters = settings['ts_adapters']
ts_adapters = [a.lower() for a in ts_adapters]

print(f'Considering the following TS Search methods: {ts_adapters}')

for method in ts_adapters:
    if method in all_families_ts_adapters or \
            (rxn.family is not None
             and rxn.family.label in list(ts_adapters_by_rmg_family.keys())
             and method in ts_adapters_by_rmg_family[rxn.family.label]):
        sched.run_job(job_type='tsg',
                      job_adapter=method,
                      reactions=[rxn],
                      tsg=tsg_index,
                      dihedral_increment=dihedral_increment,
                     )
        tsg_index += 1

print(f'Got {len(rxn.ts_species.ts_guesses)} TS Guesses (before clustering).')

rxn.ts_species.cluster_tsgs()
        
print(f'Got {len(rxn.ts_species.ts_guesses)} TS Guesses (after clustering).')

Considering the following TS Search methods: ['heuristics', 'autotst', 'gcn', 'kinbot']
dihedral_increment: 90


/bin/bash: -c: line 0: syntax error near unexpected token `('
/bin/bash: -c: line 0: `source ~/.bashrc; /home/alon/anaconda3/envs/tst_env/bin/python /mnt/Data/Code/ARC/arc/job/adapters/ts/scripts/autotst_script.py [OH]+CCC_O+[CH2]CC /home/alon/Dropbox (MIT)/runs/TS_search_1/calcs/TS_guesses/TS/tsg1/ts_results.yml'
Got return code: 1
stdout: None
stderr: None
/bin/bash: -c: line 0: syntax error near unexpected token `('
/bin/bash: -c: line 0: `source ~/.bashrc; /home/alon/anaconda3/envs/tst_env/bin/python /mnt/Data/Code/ARC/arc/job/adapters/ts/scripts/autotst_script.py O+[CH2]CC_[OH]+CCC /home/alon/Dropbox (MIT)/runs/TS_search_1/calcs/TS_guesses/TS/tsg1/ts_results.yml'
Got return code: 1
stdout: None
stderr: None


Got 4 TS Guesses (before clustering).
Got 4 TS Guesses (after clustering).


In [9]:
for tsg in rxn.ts_species.ts_guesses:
    print(f'{tsg.method} {tsg.method_index} (success: {tsg.success}):')
    print(f'coordinates:\n{xyz_to_str(tsg.initial_xyz)}')
    if tsg.initial_xyz is not None:
        draw_structure(xyz=tsg.initial_xyz)
    print('\n\n\n')

heuristics 0 (success: True):
coordinates:
O       1.43737500    1.58112542    0.11736861
H       1.91964767    0.94498355   -0.43190146
C      -0.56973933    0.33273535    0.85620513
C      -0.56973933   -1.07932962    0.29545433
C      -0.56973933   -1.07932962   -1.22387717
H       0.49495302    0.99495540    0.46428247
H      -0.56974009    0.30632916    1.95037139
H      -1.45698281    0.88458558    0.52960174
H      -1.45215391   -1.61664362    0.66021023
H       0.31267551   -1.61664346    0.66021017
H       0.31750400   -0.57210949   -1.61622422
H      -1.45698284   -0.57210981   -1.61622424
H      -0.56973903   -2.10599218   -1.60316723


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol





heuristics 1 (success: True):
coordinates:
O       1.44990567    1.56370672    0.12197742
H       1.17292977    1.98298507   -0.70654599
C      -0.55720866    0.31531664    0.86081394
C      -0.55720866   -1.09674832    0.30006314
C      -0.55720866   -1.09674832   -1.21926836
H       0.50748369    0.97753670    0.46889128
H      -0.55720941    0.28891046    1.95498020
H      -1.44445213    0.86716687    0.53421055
H      -1.43962324   -1.63406232    0.66481904
H       0.32520618   -1.63406216    0.66481898
H       0.33003468   -0.58952819   -1.61161541
H      -1.44445217   -0.58952851   -1.61161543
H      -0.55720836   -2.12341088   -1.59855842


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol





heuristics 2 (success: True):
coordinates:
O       1.44703222    1.55623294    0.10154324
H       1.34416191    2.42835646    0.51115195
C      -0.56008211    0.30784287    0.84037976
C      -0.56008211   -1.10422209    0.27962895
C      -0.56008211   -1.10422209   -1.23970255
H       0.50461024    0.97006292    0.44845710
H      -0.56008286    0.28143668    1.93454601
H      -1.44732558    0.85969310    0.51377636
H      -1.44249668   -1.64153610    0.64438485
H       0.32233273   -1.64153593    0.64438479
H       0.32716123   -0.59700196   -1.63204960
H      -1.44732561   -0.59700228   -1.63204962
H      -0.56008181   -2.13088465   -1.61899260


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol





heuristics 3 (success: True):
coordinates:
O       1.43450155    1.57365164    0.09693443
H       2.09087982    1.39035494    0.78579648
C      -0.57261278    0.32526157    0.83577095
C      -0.57261278   -1.08680339    0.27502014
C      -0.57261278   -1.08680339   -1.24431136
H       0.49207957    0.98748162    0.44384829
H      -0.57261353    0.29885538    1.92993720
H      -1.45985625    0.87711180    0.50916755
H      -1.45502736   -1.62411740    0.63977604
H       0.30980206   -1.62411723    0.63977598
H       0.31463056   -0.57958326   -1.63665841
H      -1.45985628   -0.57958358   -1.63665843
H      -0.57261248   -2.11346595   -1.62360141


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol